## A lancer sur 3D Slicer

In [9]:
import unittest 
import re
from slicer.ScriptedLoadableModule import *
import logging
from __main__ import vtk, qt, ctk, slicer
from math import *
import numpy as np
from vtk.util import numpy_support
import SimpleITK as sitk
import sitkUtils as su
import time
import datetime
import pydicom
import sys, time, os
from datetime import datetime
from PIL import Image

In [10]:
### Cropping images ###

def crop_image(image_path, NcoupeEnZ):
    image           = sitk.ReadImage(image_path)
    TailleImage     = image.GetSize()
    UpperBondingBox = [0, 0, 0] 
    LowerBondingBox = [0, 0, TailleImage[2] - NcoupeEnZ] ## 100 pour la taille de l'image que l'on souhaite
    
    image_select    = cropImagefctLabel(image, LowerBondingBox, UpperBondingBox)
    su.PushVolumeToSlicer(image_select, name = 'image_select', className = 'vtkMRMLScalarVolumeNode')
    
    return image_select

def cropImagefctLabel(image, LowerBondingBox, UpperBondingBox):
    image_cropper = sitk.Crop(image, LowerBondingBox, UpperBondingBox)
    
    return image_cropper

In [11]:
### Rééchantillonnage IRM

def reechantillonnage_IRM(image_ref, image_mobile, transformation):
    resampler = sitk.ResampleImageFilter()
    
    resampler.SetReferenceImage(image_ref)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(0)
    resampler.SetTransform(transformation)
    
    ImageRecaler = resampler.Execute(image_mobile)
    
    return ImageRecaler

In [12]:
### Rééchantillonnage CT

def reechantillonnage_CT(image_ref, image_mobile, transformation):
    resampler = sitk.ResampleImageFilter()
    
    resampler.SetReferenceImage(image_ref)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(-1000)
    resampler.SetTransform(transformation)
    
    ImageRecaler = resampler.Execute(image_mobile)
    
    return ImageRecaler

In [13]:
### Rééchantillonnage NeirestNeighbor

def reechantillonnage_CT(image_ref, image_mobile, transformation):
    resampler = sitk.ResampleImageFilter()
    
    resampler.SetReferenceImage(image_ref)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetDefaultPixelValue(0)
    resampler.SetTransform(transformation)
    
    ImageRecaler = resampler.Execute(image_mobile)
    
    return ImageRecaler

In [14]:
def multires_registrations(fixed_image, moving_image, initial_transform, ImageSamplingPercentage):
    registration_method = sitk.ImageRegistrationMethod()
    
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins = 50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(float(ImageSamplingPercentage)/100)
    registration_method.SetInterpolator(sitk.sitkLinear)
    registration_method.SetOptimizerAsGradientDescent(learningRate = 1.0, estimateLearningRate = registration_method.EachIteration, numberOfIterations = 100)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetInitialTransform(initial_transform)
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [3, 2, 1])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
    
    final_transform = registration_method.Execute(fixed_image, moving_image)
    
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    
    return final_transform

In [15]:
def main_image_Traitement_IRM(image_template_IRM, image_arecaler_IRMpost, pourcentage_image):
    image_arecaler_IRMpost = sitk.Cast(image_arecaler_IRMpost, sitk.sitkFloat32)
    image_template_IRM     = sitk.Cast(image_template_IRM, sitk.sitkFloat32)
    ####### 
    image_arecaler_IRMpost.SetDirection(image_template_IRM.GetDirection())
    image_arecaler_IRMpost.SetOrigin(image_template_IRM.GetOrigin())
    
    ### Recalage primaire de la somme des moments des images ###
    initial_transform = sitk.CenteredTransformInitializer(image_template_IRM, image_arecaler_IRMpost, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.MOMENTS)
    
    ### Recalage multiresolution rigide des images ###
    medium_transform = multires_registrations(image_template_IRM, image_arecaler_IRMpost, initial_transform, pourcentage_image)
    
    ### Rééchantillonnage ###
    image_arecaler_IRMpost = reechantillonnage_IRM(image_template_IRM, image_arecaler_IRMpost, medium_transform)
    su.PushVolumeToSlicer(image_arecaler_IRMpost, name = 'MRIregistertoCT', className = 'vtkMRMLScalarVolumeNode')

    return image_arecaler_IRMpost

In [16]:
def main_image_Traitement_CT(image_template_IRM, image_arecaler_CT, image_dose, pourcentage_image, filenameCT):
    image_arecaler_CT  = sitk.Cast(image_arecaler_CT, sitk.sitkFloat32)
    image_template_IRM = sitk.Cast(image_template_IRM, sitk.sitkFloat32)
    image_dose         = sitk.Cast(image_dose, sitk.sitkFloat32)
    
    ### Recalage primaire de la somme des moments des images ###
    initial_transform = sitk.CenteredTransformInitializer(image_template_IRM, image_arecaler_CT, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.MOMENTS)
    
    ### Recalage multiresolution rigide des images ###
    medium_transform = multires_registrations(image_template_IRM, image_arecaler_CT, initial_transform, pourcentage_image)
    
    ### Rééchantillonnage ###
    image_arecaler_CT = reechantillonnage_CT(image_template_IRM, image_arecaler_CT, medium_transform)
    image_dose        = reechantillonnage_IRM(image_template_IRM, image_dose, medium_transform)
    
    su.PushVolumeToSlicer(image_arecaler_CT, name = 'MRIregistertoCT', className = 'vtkMRMLScalarVolumeNode')
    sitk.WriteTransform(medium_transform, "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/04_Preprocessing_recal/" + filenameCT + ".h5")

    return image_arecaler_CT, image_dose

In [18]:
def main(input_data_directory, crop_data_directory, recaler_data_directory):
    '''
    Cette fonction permet :
    
    1. De cropper les IRM pré traitement et de les sauvegarder dans un fichier "Preprocessing_crop".
       
    2. De recaler l'IRM post traitement sur l'IRM pré traitement
    
    3. De recaler le CT et le RTDOSE sur l'IRM pré traitement
    '''
    
    pourcentage_image = 20         #20 pour validation
    NcoupeEnZ         = 200
    IRM_post          = 'IRMpost_4mois'
    CT                = 'CT'
    RTDose            = 'RTDOSE'

    
    for filename in os.listdir(input_data_directory):
        if 'IRMpre' in filename:
            input_image_path   = os.path.join(input_data_directory, filename)
            output_image_path  = os.path.join(crop_data_directory, filename)          
            
            image_crop         = crop_image(input_image_path, NcoupeEnZ)
            sitk.WriteImage(image_crop, output_image_path)

     
    PatientID = None
    for filename_pre in os.listdir(crop_data_directory): 
        PatientID = filename_pre.split('_')[0]
            
        filename_CT   = PatientID + '_' + CT
        filename_dose = PatientID + '_' + RTDose
        filename_post = PatientID + '_' + IRM_post 
        
        try :
            if filename_post + '.nii' in os.listdir(input_data_directory):
                input_data_directory   = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/02_Preprocessing_nii"  
                output_image_path      = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/03_Preprocessing_crop" 
            
                post_image_path        = os.path.join(input_data_directory, filename_post + '.nii')
                crop_image_path        = os.path.join(output_image_path, filename_pre)
                   
                image_recaler          = main_image_Traitement_IRM(sitk.ReadImage(crop_image_path), sitk.ReadImage(post_image_path), pourcentage_image)
                sitk.WriteImage(image_recaler, recaler_data_directory + '/' + filename_post + '_recal.nii' )
            
        except:
            print('IRM post non réalisé pour le patient', PatientID)
        
        try :
            if filename_CT + '.nii' in os.listdir(input_data_directory):
                input_data_directory   = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/02_Preprocessing_nii"  
                output_image_path      = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/03_Preprocessing_crop" 
            
                crop_image_path        = os.path.join(output_image_path, filename_pre)
                CT_image_path          = os.path.join(input_data_directory, filename_CT + '.nii')
                dose_image_path        = os.path.join(input_data_directory, filename_dose + '.nii')
            
                image_recaler_CT, image_recaler_dose = main_image_Traitement_CT(sitk.ReadImage(crop_image_path), sitk.ReadImage(CT_image_path), sitk.ReadImage(dose_image_path), pourcentage_image, filename_CT)
                sitk.WriteImage(image_recaler_CT, recaler_data_directory + '/' + filename_CT + '_recal.nii')
                sitk.WriteImage(image_recaler_dose, recaler_data_directory + '/' + filename_dose + '_recal.nii') 
                
        except :
            print('Erreur')

 
    return 0

In [19]:
### Chemins d'entrée et de sortie ###

input_data_directory   = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/02_Preprocessing_nii"  
crop_data_directory    = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/03_Preprocessing_crop"   
recaler_data_directory = "/home/aurelien/Documents/IA-Med-Im/Partie1_GBM_RT/04_Preprocessing_recal"   

In [ ]:
main(input_data_directory, crop_data_directory, recaler_data_directory)